# Generate Dataset

In [8]:
import cv2

In [ ]:
def generate_dataset():
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    if face_classifier.empty():
        print("Error: Unable to load Haarcascade file.")
        return

    def face_cropped(img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray, 1.3, 5)

        if len(faces) == 0:
            return None

        for (x, y, w, h) in faces:
            cropped_face = img[y:y+h, x:x+w]
            return cropped_face

    cap = cv2.VideoCapture(0)

    if not cap.isOpened():
        print("Error: Camera not accessible.")
        return

    id = 4
    img_id = 0

    cv2.namedWindow('Cropped Face', cv2.WINDOW_NORMAL)
    
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to capture image.")
            break

        cropped_face = face_cropped(frame)
        
        if cropped_face is not None:
            img_id += 1
            face = cv2.resize(cropped_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            path_name = 'C:/Users/HP/Documents/ML/Face Recognizer/data/user.' + str(id) + '.' + str(img_id) + '.jpg'
            cv2.imwrite(path_name, face)

            cv2.putText(face, str(img_id), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Cropped Face', face)

            print(f"Saved: {path_name}")

        else:
            print("No face detected.")

        if cv2.waitKey(1) == 13 or img_id == 200:
            break

    cap.release()
    cv2.destroyAllWindows()

generate_dataset()


# Train the classifier

In [8]:
import numpy as np 
from PIL import Image
import os
import cv2

In [ ]:
def train_classifier(data_dir):
    path = [os.path.join(data_dir, f) for f in os.listdir(data_dir)]
    faces = []
    ids = []
    
    for image in path:
        img = Image.open(image).convert('L')
        imageNp = np.array(img, 'uint8')  
        
        id = int(os.path.split(image)[1].split('.')[1])
        
        faces.append(imageNp)
        ids.append(id)
    
    ids = np.array(ids)
    
    clf = cv2.face.LBPHFaceRecognizer_create()

    clf.train(faces, ids)

    clf.write('classifier.xml')

    print("Training complete. Model saved as 'classifier.xml'.")

train_classifier('C:\\Users\\HP\\Documents\\ML\\Face Recognizer\\data')

# Detect Face

In [11]:
import cv2
import numpy as np
from PIL import Image
import os

In [ ]:
def draw_boundary(img, classifier, scaleFactor, minNeighbors, color, text, clf):
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbors)
    
    coords = []
    
    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
        id, pred = clf.predict(gray_image[y:y+h, x+w])
        confidence = int(100 * (1 - pred / 300))
        
        if confidence > 75:
            if id == 1:
                cv2.putText(img, 'Rohan', (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, color, 1, cv2.LINE_AA)
            elif id == 2:
                cv2.putText(img, 'Sara', (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, color, 1, cv2.LINE_AA)
            elif id == 3:
                cv2.putText(img, 'Jesmin', (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, color, 1, cv2.LINE_AA)
            elif id == 4:
                cv2.putText(img, 'Sarwar', (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, color, 1, cv2.LINE_AA)
        else:
            cv2.putText(img, 'Unknown', (x, y-5), cv2.FONT_HERSHEY_COMPLEX, 0.8, color, 1, cv2.LINE_AA)
            
        coords = [x, y, w, h]
        
    return coords
    
def recognize(img, clf, faceCascade):
    coords = draw_boundary(img, faceCascade, 1.1, 10, (255, 255, 255), 'Face', clf)
    return img

faceCascade = face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
clf = cv2.face.LBPHFaceRecognizer_create()
clf.read('C:\\Users\\HP\\Documents\\ML\\Face Recognizer\\classifier.xml')

video_capture = cv2.VideoCapture(0)
while True:
    ret, img = video_capture.read()
    img = recognize(img, clf, faceCascade)
    cv2.imshow('Face Detection', img)
    
    if cv2.waitKey(1) == 13:
        break
video_capture.release()
cv2.destroyAllWindows()